In [1]:
import sys   
sys.path.append('../../src/data')
import query_flights as query
path_to_db = "sqlite:///../../data/processed/airlines.db"

In [2]:
# Get Flights For A

In [4]:
# get_features.py
import pandas as pd
from datetime import date
from sqlalchemy import create_engine

def query_flights_for_date_range(start_date=None, end_date=None, 
                               airport_code=None, path_to_db=None):
    """Retrieve recorded flight data for a given time range and airport"""
    if start_date is None:
        start_date = date(2017, 1, 1)
    if end_date is None:
        end_date = date(2017, 1, 31)
    if airport_code is None:
        airport_code = 'MDT'
        
    engine = create_engine(path_to_db)
    with engine.connect() as conn:
        flights = pd.read_sql("""
            SELECT 
                *
            FROM 
                flights AS f
            WHERE 
                f.Origin = :airport_code
            AND
                f.FlightDate 
            BETWEEN :start_date 
                AND :end_date
        """, conn, 
        params={'start_date': start_date, 'end_date': end_date, 'airport_code': airport_code})
    return flights

def query_weather_for_date_range(start_date=None, end_date=None, 
                                 airport_code=None, path_to_db=None):
    """Retrieve recorded flight data for a given time range and airport"""
    if start_date is None:
        start_date = date(2017, 1, 1)
    if end_date is None:
        end_date = date(2017, 1, 31)
    if airport_code is None:
        airport_code = 'MDT'
        
    engine = create_engine(path_to_db)
    with engine.connect() as conn:
        flights = pd.read_sql("""
            SELECT 
                a.airport,
                a.display_airport_name,
                w.station,
                w.date,
                w.latitude,
                w.longitude,
                w.hourly_visibility,
                w.hourly_dry_bulb_temp_f,
                w.hourly_precipitation,
                w.hourly_wind_speed,
                w.hourly_wind_gust_speed,
                w.hourly_station_pressure
            FROM 
                weather AS w
            JOIN 
                airports as a
            ON a.int_latitude = w.int_latitude 
            AND a.int_longitude = w.int_longitude
            WHERE 
                a.airport = :airport_code
            AND 
                a.airport_is_latest == 1
            AND
                w.date 
            BETWEEN :start_date 
                AND :end_date
        """, conn, 
        params={'start_date': start_date, 'end_date': end_date, 'airport_code': airport_code})
    return flights

def get_features_for_flight_analysis(start_date=None, end_date=None, 
                                     airport_code=None, path_to_db=None):
    """Build Features for Analysis of Flights"""
    if start_date is None:
        start_date = date(2017, 1, 1)
    if end_date is None:
        end_date = date(2017, 1, 31)
    if airport_code is None:
        airport_code = 'MDT'
    
    # Get Flights For 
    #flights = query_flights_for_date_range(start_date, end_date, airport_code, path_to_db)
    
    # Get Hourly Summaries Of Airport
    # Get Hourly Summaries of Weather
    weather = query_weather_for_date_range(start_date, end_date, airport_code, path_to_db)
    
    # Extract datetime object to the hour

    return weather

flights = get_features_for_flight_analysis(path_to_db=path_to_db)
flights.head()

,airport,display_airport_name,station,date,latitude,longitude,hourly_visibility,hourly_dry_bulb_temp_f,hourly_precipitation,hourly_wind_speed,hourly_wind_gust_speed,hourly_station_pressure
